In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Generating Images using pre-trained VAE model in PyTorch
* The model training can be found here. https://www.kaggle.com/adhok93/vq-vae-training

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter
import umap
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from six.moves import xrange
import torchvision

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

## Defining the model object

In [ ]:
class VectorQuantizer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, commitment_cost):
        super(VectorQuantizer, self).__init__()
        
        ### Create an embedding matrix with size number of embedding X embedding dimension
        self._embedding_dim = embedding_dim
        self._num_embeddings = num_embeddings
        
        self._embedding = nn.Embedding(self._num_embeddings, self._embedding_dim)
        self._embedding.weight.data.uniform_(-1/self._num_embeddings, 1/self._num_embeddings)
        self._commitment_cost = commitment_cost

    def forward(self, inputs):
        # convert inputs from BCHW -> BHWC
        inputs = inputs.permute(0, 2, 3, 1).contiguous()
        input_shape = inputs.shape
        
        # Flatten input
        flat_input = inputs.view(-1, self._embedding_dim)
        
        # Calculate distances between flattened input and embedding vector
        distances = (torch.sum(flat_input**2, dim=1, keepdim=True) 
                    + torch.sum(self._embedding.weight**2, dim=1)
                    - 2 * torch.matmul(flat_input, self._embedding.weight.t()))
        
            
        # Choose indices that are min in each row
        encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
        ## Create a matrix of dimensions B*H*W into number of embeddings
        encodings = torch.zeros(encoding_indices.shape[0], self._num_embeddings, device=inputs.device)
        ### Convert index to on hot encoding 
        encodings.scatter_(1, encoding_indices, 1)
        
        # Quantize and unflatten
        quantized = torch.matmul(encodings, self._embedding.weight).view(input_shape)
        
        # Loss
        e_latent_loss = F.mse_loss(quantized.detach(), inputs)
        q_latent_loss = F.mse_loss(quantized, inputs.detach())
        loss = q_latent_loss + self._commitment_cost * e_latent_loss
        
        quantized = inputs + (quantized - inputs).detach()
        
        avg_probs = torch.mean(encodings, dim=0)
        perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))
        
        # convert quantized from BHWC -> BCHW
        return loss, quantized.permute(0, 3, 1, 2).contiguous(), perplexity, encodings
### Create Residual connections
class Residual(nn.Module):
    def __init__(self,in_channels,num_hiddens,num_residual_hiddens):
        super(Residual,self).__init__()
        self._block=nn.Sequential(
            nn.ReLU(True),
            nn.Conv2d(in_channels=in_channels,
                     out_channels=num_residual_hiddens,
                     kernel_size=3,stride=1,padding=1,bias=False),
            nn.ReLU(True),
            nn.Conv2d(in_channels=num_residual_hiddens,
                     out_channels=num_hiddens,
                     kernel_size=1,stride=1,bias=False)
        )
        
    def forward(self,x):
        return x + self._block(x)
class ResidualStack(nn.Module):
    def __init__(self,in_channels,num_hiddens,num_residual_layers,num_residual_hiddens):
        super(ResidualStack,self).__init__()
        self._num_residual_layers=num_residual_layers
        self._layers = nn.ModuleList([Residual(in_channels,num_hiddens,num_residual_hiddens) for _ in range(self._num_residual_layers)])
    def forward(self,x):
        for i in range(self._num_residual_layers):
            x=self._layers[i](x)
        return F.relu(x)

class Encoder(nn.Module):
    def __init__(self,in_channels,num_hiddens,num_residual_layers,num_residual_hiddens):
        super(Encoder,self).__init__()
        self._conv_1 = nn.Conv2d(in_channels=in_channels,
                                out_channels=num_hiddens//2,
                                kernel_size=4,
                                stride=2,padding=1)
        self._conv_2 = nn.Conv2d(in_channels=num_hiddens//2,
                                 out_channels = num_hiddens,
                                 kernel_size=4,
                                 stride=2,padding=1
                                )
        self._conv_3 = nn.Conv2d(in_channels=num_hiddens,
                                out_channels=num_hiddens,
                                kernel_size=3,
                                stride=1,padding=1)
        self._residual_stack = ResidualStack(in_channels = num_hiddens,
                                             num_hiddens = num_hiddens,
                                             num_residual_layers = num_residual_layers,
                                             num_residual_hiddens = num_residual_hiddens
                                            )
    def forward(self,inputs):
        x = self._conv_1(inputs)
        x = F.relu(x)
        x = self._conv_2(x)
        x = F.relu(x)
        x = self._conv_3(x)
        return self._residual_stack(x)
class Decoder(nn.Module):
    def __init__(self,in_channels,num_hiddens,num_residual_layers,num_residual_hiddens):
        super(Decoder,self).__init__()
        self._conv_1 = nn.Conv2d(in_channels=in_channels,
                                out_channels= num_hiddens,
                                kernel_size=3,
                                stride=1,padding=1)
        self._residual_stack = ResidualStack(in_channels=num_hiddens,
                                             num_hiddens=num_hiddens,
                                             num_residual_layers=num_residual_layers,
                                             num_residual_hiddens= num_residual_hiddens
                                            )
        self._conv_trans_1 = nn.ConvTranspose2d(in_channels=num_hiddens,
                                               out_channels=num_hiddens//2,
                                               kernel_size=4,
                                               stride=2,padding=1)
        self._conv_trans_2 = nn.ConvTranspose2d(in_channels=num_hiddens//2,
                                               out_channels=3,
                                               kernel_size=4,
                                               stride=2,padding=1)
    def forward(self,inputs):
        x = self._conv_1(inputs)
        x = self._residual_stack(x)
        x = self._conv_trans_1(x)
        x = F.relu(x)
        return self._conv_trans_2(x)
class Model(nn.Module):
    def __init__(self,num_hiddens,num_residual_layers,num_residual_hiddens,num_embeddings,embedding_dim,commitment_cost,decay=0):
        super(Model,self).__init__()
        self._encoder_= Encoder(3,num_hiddens,num_residual_layers,num_residual_hiddens)
        self._pre_vq_conv = nn.Conv2d(in_channels=num_hiddens,
                                     out_channels=embedding_dim,
                                     kernel_size=1,
                                     stride=1)
        self._vq_vae = VectorQuantizer(num_embeddings,embedding_dim,commitment_cost)
        self._decoder = Decoder(embedding_dim,
                              num_hiddens,
                              num_residual_layers,
                              num_residual_hiddens)
    def forward(self,x):
        z = self._encoder_(x)
        z = self._pre_vq_conv(z)
        loss,quantized,perplexity,_ = self._vq_vae(z)
        x_recon = self._decoder(quantized)
        return loss,x_recon,perplexity
            
        
num_training_updates = 150
num_hiddens = 128
num_residual_hiddens = 32
num_residual_layers = 3
embedding_dim= 512
num_embeddings = 512
commitment_cost = 0.25
learning_rate = 3e-4
model = Model(num_hiddens,num_residual_layers,num_residual_hiddens,num_embeddings,embedding_dim,commitment_cost,decay=0)

## Importing the pre-trained model weights

In [ ]:
def load_checkpoint(filepath):
        checkpoint = torch.load(filepath)
        model = checkpoint['model']
        model.load_state_dict(checkpoint['state_dict'])
        for parameter in model.parameters():
            parameter.requires_grad = False

        model.eval()

        return model



model = load_checkpoint('../input/checkpoint-new-vae/checkpoint_new.pth')

## Getting the dataset

In [ ]:
train_data_path = '../input/gan-getting-started/'
 
### Rescaling incoming image to 28 by 28 pixels
### After Rescaling, convert the image to a tensor
transform = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5), (1.0,1.0,1.0))
                               ])
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=transform)
train_loader = torch.utils.data.DataLoader(train_data,5,shuffle=True)

In [ ]:
# model.eval()
# import matplotlib.pyplot as plt
# from PIL import ImageFilter
# from random import randrange
# import random



# trans = transforms.ToPILImage()
# for i in range(100):
#     second_dim = randrange(512)
#     third_dim = randrange(8)
#     fourth_dim = randrange(8)
#     data,_ = next(iter(train_loader))
#     encoder_output = model._encoder_(data)
#     loss,quantized,perplexity,_ = model._vq_vae(model._pre_vq_conv(encoder_output))
#     #quantized[:,second_dim,third_dim,fourth_dim] = quantized[:,second_dim,third_dim,fourth_dim]+random.random()
#     recon = model._decoder(quantized)
#     recon = recon.view(3,32,32)
#     recon = trans(recon).resize((32,32))


#     plt.imshow(np.asarray(recon))
#     plt.pause(0.5)
#     print('---')
#     plt.imshow(np.asarray(trans(data.view(3,32,32))))
#     plt.pause(0.5)

In [ ]:


# model.eval()
# second_dim = randrange(512)
# third_dim = randrange(8)
# fourth_dim = randrange(8)

# (valid_originals, _) = next(iter(train_loader))
# valid_originals = valid_originals.to(device)

# vq_output_eval = model._pre_vq_conv(model._encoder_(valid_originals))
# _, valid_quantize, _, _ = model._vq_vae(vq_output_eval)
# valid_reconstructions = model._decoder(valid_quantize)
# show(make_grid(valid_reconstructions.cpu().data+0.5), )



## Adding Gaussian Noise to a batch of embeddings

* VQ-VAE outputs seem to be a bit blurry due to added noise

In [ ]:
from random import randrange
from torch.distributions.uniform import Uniform
from torch.distributions.normal import Normal
from PIL import Image


import random
def show(img):
    npimg = img.numpy()
    fig = plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.pause(0.5)

## Take a random single batch

for i in range(randrange(20)):
    
    (valid_originals, _) = next(iter(train_loader))


for i in range(20):
    

    model.eval()
    second_dim = randrange(512)
    third_dim = randrange(8)
    fourth_dim = randrange(8)

    valid_originals = valid_originals.to(device)

    vq_output_eval = model._pre_vq_conv(model._encoder_(valid_originals))
    _, valid_quantize, _, _ = model._vq_vae(vq_output_eval)
    #valid_quantize[:,second_dim,third_dim,:] = valid_quantize[:,second_dim,third_dim,:]+random.random()
    #valid_quantize = torch.tensor(np.random.rand(10,512,8,8)).float()
    shape = 5,512,8,8
    noise = Normal(0,1).sample(shape)
    valid_quantize = valid_quantize+noise
    valid_reconstructions = model._decoder(valid_quantize)
    

#     trans = transforms.ToPILImage()
#     for j in range(5):
#         valid_image = valid_reconstructions[i,:,:,:]
#         valid_image = trans(valid_image)
#         name = 'image_'+str(i)+str(j)+'.jpg'
#         valid_image.save(name, "JPEG")
    show(make_grid(valid_reconstructions.cpu().data+0.5), )
